# About Notebook

It is basic starter notebook for Stable-Diffusion Image-to-Prompt, written in `keras`, `keras-cv`, and `transformer` from huggingface. It contains both **training code** and **inference code**. Please note, the training code is configured on **TPU-VM**, otherwise on **GPU**.


In [1]:
# set True for 'Inference' on GPU (turn off the internet)
# set False for 'Training' on TPU-VM (turn on the internet)
SUBMIT = False

In [2]:
import os
import warnings
import logging
from IPython.display import clear_output
warnings.filterwarnings('ignore')
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
logging.disable(logging.WARNING)


if SUBMIT:
    !pip install --no-deps ../input/keras-cv/keras_cv-0.4.2-py3-none-any.whl
else:
    !pip install -U -q scikit-learn
    !pip install -U -q transformers
    
clear_output()

In [3]:
import os
import glob
import numpy as np
import pandas as pd 
import warnings
import random
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import callbacks
from tensorflow.keras.optimizers import schedules
from tensorflow.python.client import device_lib

try:
    import keras_cv
    from keras_cv.layers import RandomBrightness
    from keras_cv.layers import RandomChannelShift
    from keras_cv.layers import RandomFlip
except:
    from keras.layers import RandomFlip

import transformers
from transformers import AutoConfig
from transformers import TFBertTokenizer
from transformers import TFAutoModel
from transformers import ViTFeatureExtractor
from transformers import TFViTModel

transformers.logging.disable_progress_bar()
clear_output()

# Devices

In [4]:
if SUBMIT:
    physical_devices = tf.config.list_physical_devices('GPU')
    tf.config.optimizer.set_jit(True)
    keras.mixed_precision.set_global_policy("mixed_float16")
    [tf.config.experimental.set_memory_growth(pd, True) for pd in physical_devices]
    strategy = tf.distribute.MirroredStrategy()
else:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect(tpu="local")
    strategy = tf.distribute.TPUStrategy(tpu)
    keras.mixed_precision.set_global_policy("mixed_bfloat16")

    
seed = 42
input_size = 224
batch_size = 32 * strategy.num_replicas_in_sync
num_epochs = 10
keras.utils.set_random_seed(seed)
tf.__version__, transformers.__version__

('2.12.0', '4.28.1')

# Data [Training]

In [5]:
df = pd.read_csv('/kaggle/input/diffusiondb-data-cleansing/diffusiondb.csv')
trn_df, val_df = train_test_split(
    df, test_size=0.1, random_state=seed
)
trn_df.shape, val_df.shape

((138888, 2), (15432, 2))

In [6]:
# get max sequence
trn_df["prompt"].apply(lambda x: len(x.split())).describe()

count    138888.000000
mean         13.130083
std          10.864133
min           5.000000
25%           7.000000
50%          10.000000
75%          15.000000
max         480.000000
Name: prompt, dtype: float64

# Dataloader [Training]

In [7]:
model_id = 'sentence-transformers/all-MiniLM-L6-v2'
tokenizer = TFBertTokenizer.from_pretrained(model_id)

def read_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.io.decode_png(image, 3)
    image = tf.image.resize(image, (input_size, input_size))
    return image

def batch_tokenize(prompt):
    tokenized = tokenizer(
        prompt, 
        padding="max_length", 
        truncation=True, 
        max_length=128
    )
    return tokenized

In [8]:
def dataloader(
    image_paths, 
    prompts, 
    shuffle=True, 
    repeat=True, 
    batch_size=1,
):
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, prompts))
    dataset = dataset.map(
        lambda x, y: (read_image(x), y),
        num_parallel_calls=tf.data.AUTOTUNE
    )
    
    dataset = dataset.batch(
        batch_size, drop_remainder=True
    )

    dataset = dataset.map(
        lambda x, y: (x, (batch_tokenize(y))),
        num_parallel_calls=tf.data.AUTOTUNE
    )
    
    if shuffle:
        dataset = dataset.shuffle(
            batch_size * 8, reshuffle_each_iteration = False
        )
    
    if repeat:
        dataset = dataset.repeat()
        
    return dataset.prefetch(tf.data.AUTOTUNE)

In [9]:
train_ds = dataloader(
    trn_df["filepath"].values, 
    trn_df['prompt'].values, 
    shuffle=True, 
    repeat=False,
    batch_size=batch_size
)

valid_ds = dataloader(
    val_df["filepath"].values, 
    val_df['prompt'].values, 
    shuffle=False, 
    repeat=False, 
    batch_size=batch_size
)

# Model

In [10]:
def preprocess(x):
    x = keras.layers.Rescaling(scale=1./127.5, offset=-1.0)(x)
    x = keras.layers.Permute(dims=(3, 1, 2))(x)
    return x
    
def augment(x):
    try:
        pipeline = keras_cv.layers.RandomChoice(
            layers=[
                RandomFlip("horizontal")
            ],
            auto_vectorize=True
        )
    except:
        pipeline = keras.Sequential(
            [
                keras.layers.RandomFlip("horizontal")
            ]
        )
    return pipeline(x)

In [11]:
def get_model(mode='inference'):
    input = keras.Input(shape=(input_size, input_size, 3))
    x = augment(input)
    x = preprocess(x)

    # vit model
    if mode=='inference':
        config =  AutoConfig.from_pretrained(
            '/kaggle/input/cmp-stablediffusion-imgtopt', 
            local_files_only=True
        )
        hf_model = TFViTModel.from_config(config)
        x = hf_model(x)
    else:
        x = TFViTModel.from_pretrained(
            'google/vit-base-patch16-224'
        )(x)
    x = keras.layers.Lambda(lambda v: v[:, 0])(x.last_hidden_state)
    
    # output
    output = keras.layers.Dense(
        384, activation=None, dtype=tf.float32
    )(x)

    model = keras.Model(input, output)
    return model

**Sentence Transformer (all-MiniLM-L6-v2)**

It will be run on inside the model.

In [12]:
# ref. https://www.philschmid.de/tensorflow-sentence-transformers
class TFSentenceTransformer(keras.layers.Layer):
    def __init__(self,  model_id, **kwargs):
        super().__init__(**kwargs)
        self.st_model = TFAutoModel.from_pretrained(model_id, **kwargs)

    def call(self, inputs, normalize=True):
        model_output = self.st_model(inputs)
        embeddings = self.mean_pooling(
            model_output, inputs["attention_mask"]
        )
        if normalize:
            embeddings = self.normalize(embeddings)
        return embeddings

    def mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0]
        input_mask_expanded = tf.cast(
            tf.broadcast_to(
                tf.expand_dims(attention_mask, -1), 
                tf.shape(token_embeddings)
            ),
            token_embeddings.dtype
        )
        token_mask = tf.reduce_sum(
            token_embeddings * input_mask_expanded, axis=1
        )
        mask_clip = tf.clip_by_value(
            tf.reduce_sum(input_mask_expanded, axis=1), 
            1e-9, 
            tf.float32.max
        )
        return token_mask / mask_clip
        
    def normalize(self, embeddings):
        embeddings, _ = tf.linalg.normalize(
            embeddings, 2, axis=1
        )
        return embeddings

In [13]:
class TextTokenToEmbedding(keras.Model):
    '''Transform the text/prompt-token gt to 384-D embedding with sentence transformer.
    '''
    def __init__(self, model, **kwargs):
        super().__init__(**kwargs)
        self.model = model 
    
    def call(self, inputs):
        return self.model(inputs)
    
    def train_step(self, data):
        x, y = data
        y = tfst_model(y, normalize=False)
        return super().train_step((x, y))
    
    def test_step(self, data):
        x, y = data
        y = tfst_model(y, normalize=False)
        return super().test_step((x, y))
    
    # kaggle.com/code/ipythonx/keras-rsna-breast-cancer-detection
    def save_weights(
        self, filepath, overwrite=True, save_format=None, options=None
    ):
        # Overriding this method will allow us to use the `ModelCheckpoint`
        self.model.save_weights(
            filepath=filepath,
            overwrite=overwrite,
            save_format=save_format,
            options=options,
        )
        
    def save(
        self, filepath, overwrite=True, include_optimizer=True, 
        save_format=None, signatures=None, options=None
    ):
        # Overriding this method will allow us to use the `ModelCheckpoint`
        self.model.save(
            filepath=filepath,
            overwrite=overwrite,
            save_format=save_format,
            options=options,
            include_optimizer=include_optimizer,
            signatures=signatures
        )

In [14]:
# https://stackoverflow.com/a/44933346/9215780
cosine_similarity_loss = keras.losses.CosineSimilarity(
    reduction='none'
)

def CosineEmbeddingLoss(margin=0., target=1):
    def cosine_embedding_loss_fn(input_one, input_two):
        similarity = - cosine_similarity_loss(input_one, input_two)
        return tf.reduce_mean(
            tf.where(
                tf.equal(target, 1),
                1. - similarity,
                tf.maximum(
                    tf.zeros_like(similarity), similarity - margin
                )
            )
        )
    return cosine_embedding_loss_fn

In [15]:
keras.backend.clear_session()

with strategy.scope():
    if SUBMIT:
        model = get_model(mode='inference')
        model.load_weights(
            '/kaggle/input/cmp-stablediffusion-imgtopt/model.02-0.533.h5'
        )
        model.compile(jit_compile=True)
        model.trainable = False
    else:
        # token to text-embedding
        tfst_model = TFSentenceTransformer(model_id)
        tfst_model.trainable = False

        model = get_model(mode='training')
        model = TextTokenToEmbedding(model)
        
        BASE_LR = 0.005 * batch_size / 16
        lr_decay = keras.optimizers.schedules.PiecewiseConstantDecay(
            boundaries=[12000 * 16, 16000 * 16],
            values=[BASE_LR, 0.1 * BASE_LR, 0.01 * BASE_LR],
        )
        optim = keras.optimizers.SGD(
            learning_rate=lr_decay, momentum=0.9, global_clipnorm=10.0
        )

        model.compile(
            optimizer=optim,
            loss=CosineEmbeddingLoss(margin=0., target=1),
            metrics=[
                metrics.CosineSimilarity(name='cos')
            ],
        )
        model.build(
            input_shape=(None, input_size, input_size, 3)
        )
    
clear_output()

In [16]:
model.summary(
    expand_nested=True, 
    line_length=100, 
    show_trainable=True
)

Model: "text_token_to_embedding"
_______________________________________________________________________________________________________________
 Layer (type)                                Output Shape                            Param #        Trainable  
 model (Functional)                          (None, 384)                             86684544       Y          
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| input_1 (InputLayer)                      [(None, 224, 224, 3)]                   0              Y          |
|                                                                                                             |
| sequential (Sequential)                   (None, 224, 224, 3)                     0              Y          |
||¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯||
|| random_flip (RandomFlip)                (None, 224, 224, 3)         

In [17]:
if SUBMIT:
    training_log = pd.read_csv('/kaggle/input/cmp-stablediffusion-imgtopt/training_log.csv')
    display(training_log.head())
else:
    model.fit(
        train_ds, 
        validation_data=valid_ds, 
        epochs=num_epochs,
        callbacks=[
            callbacks.ModelCheckpoint(
                filepath='model.{epoch:02d}-{val_cos:.3f}.h5',
                monitor='val_cos',
                mode='max',
                save_best_only=True
            ),
            callbacks.CSVLogger('training_log.csv')
        ]
    )

Epoch 1/10
542/542 [==============================] - 330s 271ms/step - loss: 0.5968 - cos: 0.4032 - val_loss: 0.5519 - val_cos: 0.4481
Epoch 2/10
542/542 [==============================] - 152s 228ms/step - loss: 0.5335 - cos: 0.4665 - val_loss: 0.5251 - val_cos: 0.4749
Epoch 3/10
542/542 [==============================] - 152s 228ms/step - loss: 0.5135 - cos: 0.4865 - val_loss: 0.5111 - val_cos: 0.4889
Epoch 4/10
542/542 [==============================] - 152s 228ms/step - loss: 0.5009 - cos: 0.4991 - val_loss: 0.5017 - val_cos: 0.4983
Epoch 5/10
542/542 [==============================] - 151s 228ms/step - loss: 0.4915 - cos: 0.5085 - val_loss: 0.4947 - val_cos: 0.5053
Epoch 6/10
542/542 [==============================] - 152s 229ms/step - loss: 0.4840 - cos: 0.5160 - val_loss: 0.4894 - val_cos: 0.5106
Epoch 7/10
542/542 [==============================] - 151s 227ms/step - loss: 0.4776 - cos: 0.5224 - val_loss: 0.4849 - val_cos: 0.5151
Epoch 8/10
542/542 [============================

# Data [Inference]

In [18]:
comp_path = '/kaggle/input/stable-diffusion-image-to-prompts'
imag_path = glob.glob(f'{comp_path}/images/*.png')
images = os.listdir(os.path.join(comp_path, 'images'))
imgIds = [i.split('.')[0] for i in images]

EMBEDDING_LENGTH = 384
eIds = list(range(EMBEDDING_LENGTH))

imgId_eId = [
    '_'.join(map(str, i)) for i in zip(
        np.repeat(imgIds, EMBEDDING_LENGTH),
        np.tile(range(EMBEDDING_LENGTH), len(imgIds)))
]

# Dataloader [Inference]

In [19]:
def dataloader(
    image_paths, batch_size=1
):
    dataset = tf.data.Dataset.from_tensor_slices(
        (image_paths)
    )
    dataset = dataset.map(read_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size, drop_remainder=False)
    return dataset.prefetch(tf.data.AUTOTUNE)

In [20]:
test_ds = dataloader(
    imag_path, batch_size=batch_size
)

# Prediction

In [21]:
prompt_embeddings = model.predict(test_ds)
prompt_embeddings = np.vstack(prompt_embeddings).flatten()

1/1 [==============================] - 14s 14s/step


In [22]:
submission = pd.DataFrame(
    index=imgId_eId,
    data=prompt_embeddings,
    columns=['val']
).rename_axis('imgId_eId')
submission.to_csv('submission.csv')
submission.head()

,val
imgId_eId,
f27825b2c_0,-0.572822
f27825b2c_1,0.877553
f27825b2c_2,-0.585785
f27825b2c_3,0.817140
f27825b2c_4,-0.840025
